In [3]:
# Testing deconvolutions for temp test folder (containing 1 image)

from nrrd import read, write
from tifffile import imwrite
import shutil, os
import time

# delete old directory
try:
    shutil.rmtree(r"E:\TMP\Aidan\out_test_single\temp")
    os.remove(r"E:\TMP\Aidan\out_test_single\y00006400.x00019200.nrrd")
except FileNotFoundError:
    print("File not found")
    
print("Old directory deleted")
time.sleep(2)

# run fnt_cube_processor.py
! python .\fnt_cube_processor.py -i E:\TMP\Aidan\tmp -o E:\TMP\Aidan\out_test_single -n 1 -d  
# time.sleep(2)
img, header = read(r"E:\TMP\Aidan\out_test_single\y00006400.x00019200.nrrd")
imwrite(r"E:\TMP\Aidan\out_test_single\test_decon.tif", img, compression=1)

File not found
Old directory deleted
Two objects are distinguishable in xy-plane if they are 801 nm apart. The camera pixel size is 400 nm.
Two objects are distinguishable in z-axis if they are 9272 nm apart. The z-step is 800 nm.
full width half maxima of xy-plane is 1026.4 nm.
full width half maxima of z-axis is 10251.1 nm.
E:\TMP\Aidan\out_test_single
E:\TMP\Aidan\out_test_single\temp\psf.tif
dxy_psf: 266.875
Deconvolution args:
{'otf': 'E:\\TMP\\Aidan\\out_test_single\\temp\\otf.tif', 'n_iters': 10, 'dz_data': 0.8, 'dx_data': 0.4, 'dz_psf': 0.8, 'dxy_psf': 0.266875, 'background': 0, 'wavelength_em': 525, 'na': 0.4, 'nimm': 1.52}
callednx=27, ny=9, nz=9
Center of mass is (13.000, 4.000, 4.600)
Background is -0.000
In radialft()
X_k allocated.                62MB   44021MB free Pinning raw.data's Host RAM.  Copy raw.data to X_k HostToDevice.  Done.  
CC allocated.                 62MB   43957MB free
rawGPUbuf allocated.          62MB   43893MB free
X_kminus1 allocated.          62MB 


FNT Cube Processor: 100%|##########| 1/1 [00:05<00:00,  5.17s/ cubes]


In [1]:
def convert_file(input_nrrd):
    img, header = read(input_nrrd.__str__())
    tif_path = Path(output_tif_folder) / (input_nrrd.name[:-4] + 'tif')
    imwrite(tif_path.__str__(), img, compression=1)
    return 1

In [1]:
# Testing deconvolutions for real folder (this one takes a long time)
from nrrd import read
from tifffile import imwrite
from pathlib import Path
from os import system 
from multiprocessing import Pool


# running fnt_cube_processor with 8 threads
input_folder = r"E:\TMP\Aidan\z00003200"
output_nrrd_folder = r"E:\TMP\Aidan\z00003200_deconv_nrrd"
output_tif_folder = r"E:\TMP\Aidan\z00003200_tif"
num_threads = 8

system(fr"python .\fnt_cube_processor.py -i {input_folder} -o {output_nrrd_folder} -n {num_threads} -d")

# convert to tif files
nrrd_paths = [f for f in Path(output_nrrd_folder).iterdir() if f.is_file()]

# convert single file to tif
def convert_file(input_nrrd: Path):
    img, header = read(input_nrrd.__str__())
    tif_path = Path(output_tif_folder) / (input_nrrd.name[:-4] + 'tif')
    imwrite(tif_path.__str__(), img, compression=1)
    return 1

for path in nrrd_paths:
    convert_file(path)

print("All nrrd -> tif conversions completed successfully")


All nrrd -> tif conversions completed successfully
